In [3]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import csv

#Determine current NFL week (limits the scope of data to gather)
current = input('What is the most recently completed Week of the NFL Schedule?...')
current = int(current)

players = []
pos_raw = []
pos = []
team = []
week_play = []
fan_pts = []
passyd = []
passtd = []
passint = []
rushyd = []
rushtd = []
recep = []
recyd = []
rectd = []
fumtd = []
two_conv = []
fumlost = []

url_template = (
    'https://fantasy.nfl.com/league/379759/players?offset={offset}'
    '&playerStatus=all&position=0&sort=pts&sortOrder=desc&statCategory=stats'
    '&statSeason=2019&statType=weekStats&statWeek={week}'
)

data_pts = 400

print('Begin gathering data...')

for week in range(1,current+1,1):

    # Start at 1 and count to 100 in steps of size 25 (1, 26, 51, etc.)
    # I use 138 because that appears to be the number of items in the list
    # when I went to the web site.
    for offset in range(1, data_pts, 25):
        page = requests.get(url_template.format(offset=offset,week=week))
        soup = BeautifulSoup(page.content, 'html.parser')
        tb = soup.find('table')

        # zip will terminate when the shortest iterator ends.
        for link, loop in zip(tb.find_all('tr'), range(1)):
            for name in tb.find_all('a', class_='playerName'):
                players.append(name.string)
            for position in tb.find_all('em'):
                pos_raw.append(position.string)
            for stat in tb.find_all('span', class_='statId-5'):
                passyd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-6'):
                passtd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-7'):
                passint.append(stat.string)
            for stat in tb.find_all('span', class_='statId-14'):
                rushyd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-15'):
                rushtd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-20'):
                recep.append(stat.string)
            for stat in tb.find_all('span', class_='statId-21'):
                recyd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-22'):
                rectd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-29'):
                fumtd.append(stat.string)
            for stat in tb.find_all('span', class_='statId-32'):
                two_conv.append(stat.string)
            for stat in tb.find_all('span', class_='statId-30'):
                fumlost.append(stat.string)
   
    #Create list assigning what week was played for each sample in df
    week_play = week_play + ([week] * data_pts)

pos_raw = [item for item in pos_raw if item != None]

for i in range(0, len(pos_raw)):
    part1, junk, part2 = pos_raw[i].partition('-')
    pos.append(part1.strip())
    team.append(part2.strip())
   
for i in range(0, len(players)):
    passyd[i] = int(passyd[i]) if (passyd[i] != '-') else 0
    passtd[i] = int(passtd[i]) if (passtd[i] != '-') else 0
    passint[i] = int(passint[i]) if (passint[i] != '-') else 0
    rushyd[i] = int(rushyd[i]) if (rushyd[i] != '-') else 0
    rushtd[i] = int(rushtd[i]) if (rushtd[i] != '-') else 0
    recep[i] = int(recep[i]) if (recep[i] != '-') else 0
    recyd[i] = int(recyd[i]) if (recyd[i] != '-') else 0
    rectd[i] = int(rectd[i]) if (rectd[i] != '-') else 0
    fumtd[i] = int(fumtd[i]) if (fumtd[i] != '-') else 0
    two_conv[i] = int(two_conv[i]) if (two_conv[i] != '-') else 0
    fumlost[i] = int(fumlost[i]) if (fumlost[i] != '-') else 0
    score = passyd[i]*.04 + passtd[i]*4 - passint[i]*2 + rushyd[i]*.1 + rushtd[i]*6 + recep[i]*1 + recyd[i]*.1 + rectd[i]*6 + fumtd[i]*6 + two_conv[i]*2 - fumlost[i]*2
    fan_pts.append(score)
   
print('Extracting data...')
   
df = pd.DataFrame({'Player': players,'Position': pos,'Team': team,'Week': week_play,'Fantasy Points': fan_pts,
     'Pass Yd': passyd,'Pass TD': passtd,'Pass INT': passint,'Rush Yd': rushyd,'Rush TD': rushtd,'Receptions': recep,
     'Rec Yd': recyd,'Rec TD': rectd,'Fum TD': fumtd,'2PT Conv': two_conv,'Fum Lost': fumlost})


score_blank = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
teams = ["ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN", "DET", "GB", "HOU", "IND", "JAX", "KC",
         "LA", "LAC", "MIA", "MIN", "NE", "NO", "NYG", "NYJ", "OAK", "PHI", "PIT", "SF", "SEA", "TB", "TEN", "WAS"]

#Create blank points-scored dataframe
qb_s = pd.DataFrame({'Team': teams})
rb_s = pd.DataFrame({'Team': teams})
wr_s = pd.DataFrame({'Team': teams})
te_s = pd.DataFrame({'Team': teams})
for j in range (1,current+1):
    qb_s[str(j)] = score_blank
    rb_s[str(j)] = score_blank
    wr_s[str(j)] = score_blank
    te_s[str(j)] = score_blank

#Fill in points-scored dataframe
for j in range(1,current+1):
    for i in range(0,32):
        qb_s[str(j)][i] = round(sum(df[(df['Team'] == qb_s['Team'][i]) & (df['Position'] == 'QB') & (df['Week'] == (j))]['Fantasy Points']),2)
        rb_s[str(j)][i] = round(sum(df[(df['Team'] == rb_s['Team'][i]) & (df['Position'] == 'RB') & (df['Week'] == (j))]['Fantasy Points']),2)
        wr_s[str(j)][i] = round(sum(df[(df['Team'] == wr_s['Team'][i]) & (df['Position'] == 'WR') & (df['Week'] == (j))]['Fantasy Points']),2)
        te_s[str(j)][i] = round(sum(df[(df['Team'] == te_s['Team'][i]) & (df['Position'] == 'TE') & (df['Week'] == (j))]['Fantasy Points']),2)
                         
#Upload schedule from .csv file, read as dataframe
schedule = pd.read_csv("NFL Schedule.csv")
schedule = schedule.replace({'@':''}, regex=True)
byelist = [0]*32
games_played = [0]*32
#Find team BYE weeks and calculate games played for current week
for j in range (1,18):
    for i in range (0,32):
        if schedule[str(j)][i] == 'BYE': byelist[i] = j
        games_played[i] = current if byelist[i] > current else current-1
#Add BYE and Games Played to schedule dataframe
schedule['BYE'] = byelist
schedule['Games Played'] = games_played

#Create blank points-against dataframe
qb_a = pd.DataFrame({'Team': teams})
rb_a = pd.DataFrame({'Team': teams})
wr_a = pd.DataFrame({'Team': teams})
te_a = pd.DataFrame({'Team': teams})
for j in range (1,current+1):
    qb_a[str(j)] = score_blank
    rb_a[str(j)] = score_blank
    wr_a[str(j)] = score_blank
    te_a[str(j)] = score_blank


#Fill in points-against dataframe
for j in range(1,current+1):
    for i in range(0,32):
        qb_a[str(j)][i] = 0 if schedule[str(j)][i] == 'BYE' else qb_s[str(j)][qb_s['Team'] == schedule[str(j)][i]].item()
        rb_a[str(j)][i] = 0 if schedule[str(j)][i] == 'BYE' else rb_s[str(j)][rb_s['Team'] == schedule[str(j)][i]].item()
        wr_a[str(j)][i] = 0 if schedule[str(j)][i] == 'BYE' else wr_s[str(j)][wr_s['Team'] == schedule[str(j)][i]].item()
        te_a[str(j)][i] = 0 if schedule[str(j)][i] == 'BYE' else te_s[str(j)][te_s['Team'] == schedule[str(j)][i]].item()

#Create lists with week-by-week data to sum into df_tot
qba_temp = []
rba_temp = []
wra_temp = []
tea_temp = []
qbs_temp = []
rbs_temp = []
wrs_temp = []
tes_temp = []
for i in range(0,32):
    qba_junk = []
    rba_junk = []
    wra_junk = []
    tea_junk = []
    qbs_junk = []
    rbs_junk = []
    wrs_junk = []
    tes_junk = []
    for j in range(1,current+1):
        qba_junk.append(qb_a[str(j)][i])
        rba_junk.append(rb_a[str(j)][i])
        wra_junk.append(wr_a[str(j)][i])
        tea_junk.append(te_a[str(j)][i])
        qbs_junk.append(qb_s[str(j)][i])
        rbs_junk.append(rb_s[str(j)][i])
        wrs_junk.append(wr_s[str(j)][i])
        tes_junk.append(te_s[str(j)][i])
        #Temporary dataframe to use sum() function
        df_junk = pd.DataFrame({'qba':qba_junk,'rba':rba_junk,'wra':wra_junk,'tea':tea_junk,
                                'qbs':qbs_junk,'rbs':rbs_junk,'wrs':wrs_junk,'tes':tes_junk})
    qba_temp.append(sum(df_junk['qba']))
    rba_temp.append(sum(df_junk['rba']))
    wra_temp.append(sum(df_junk['wra']))
    tea_temp.append(sum(df_junk['tea']))
    qbs_temp.append(sum(df_junk['qbs']))
    rbs_temp.append(sum(df_junk['rbs']))
    wrs_temp.append(sum(df_junk['wrs']))
    tes_temp.append(sum(df_junk['tes']))

print('Final Calculations...')

#Create dataframe with totals
df_tot = pd.DataFrame(
    {'Team': teams,'QB-A': qba_temp,'RB-A': rba_temp,'WR-A': wra_temp,'TE-A': tea_temp,
     'QB-S': qbs_temp,'RB-S': rbs_temp,'WR-S': wrs_temp,'TE-S': tes_temp, 'Games':games_played})


#Calculate Avg-Allowed for each team
df_tot['Av QB-A'] = round(df_tot['QB-A'] / df_tot['Games'],2)
df_tot['Av RB-A'] = round(df_tot['RB-A'] / df_tot['Games'],2)
df_tot['Av WR-A'] = round(df_tot['WR-A'] / df_tot['Games'],2)
df_tot['Av TE-A'] = round(df_tot['TE-A'] / df_tot['Games'],2)

#Calculate Avg-Scored for each team
df_tot['Av QB-S'] = round(df_tot['QB-S'] / df_tot['Games'],2)
df_tot['Av RB-S'] = round(df_tot['RB-S'] / df_tot['Games'],2)
df_tot['Av WR-S'] = round(df_tot['WR-S'] / df_tot['Games'],2)
df_tot['Av TE-S'] = round(df_tot['TE-S'] / df_tot['Games'],2)


#Find Opponents Avg-Scored each week (1,2,3...)
op_qb = [0]*32
op_rb = [0]*32
op_wr = [0]*32
op_te = [0]*32
for j in range(1,current+1):
    for i in range(0,32):
        op_qb[i] = df_tot['Av QB-S'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op qb-s '+ str(j)] = op_qb
        op_rb[i] = df_tot['Av RB-S'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op rb-s '+ str(j)] = op_rb
        op_wr[i] = df_tot['Av WR-S'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op wr-s '+ str(j)] = op_wr
        op_te[i] = df_tot['Av TE-S'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op te-s '+ str(j)] = op_te

#Find Opponents Avg-Allowed each week (1,2,3...)
op_qb = [0]*32
op_rb = [0]*32
op_wr = [0]*32
op_te = [0]*32

for j in range(1,current+1):
    for i in range(0,32):
        op_qb[i] = df_tot['Av QB-A'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op qb-a '+ str(j)] = op_qb
        op_rb[i] = df_tot['Av RB-A'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op rb-a '+ str(j)] = op_rb
        op_wr[i] = df_tot['Av WR-A'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op wr-a '+ str(j)] = op_wr
        op_te[i] = df_tot['Av TE-A'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op te-a '+ str(j)] = op_te

#Calculate Mean Avg-Scored for all opponents
op_qb = [0]*32
op_rb = [0]*32
op_wr = [0]*32
op_te = [0]*32
for j in range(1,current+1):
    for i in range(0,32):
        op_qb[i] = op_qb[i] + round(df_tot['op qb-s '+ str(j)][i]/current,2)
        op_rb[i] = op_rb[i] + round(df_tot['op rb-s '+ str(j)][i]/current,2)
        op_wr[i] = op_wr[i] + round(df_tot['op wr-s '+ str(j)][i]/current,2)
        op_te[i] = op_te[i] + round(df_tot['op te-s '+ str(j)][i]/current,2)
df_tot['av op qb-s'] = op_qb
df_tot['av op rb-s'] = op_rb
df_tot['av op wr-s'] = op_wr
df_tot['av op te-s'] = op_te

#Calculate Mean Avg-Allowed for all opponents
op_qb = [0]*32
op_rb = [0]*32
op_wr = [0]*32
op_te = [0]*32
for j in range(1,current+1):
    for i in range(0,32):
        op_qb[i] = op_qb[i] + round(df_tot['op qb-a '+ str(j)][i]/current,2)
        op_rb[i] = op_rb[i] + round(df_tot['op rb-a '+ str(j)][i]/current,2)
        op_wr[i] = op_wr[i] + round(df_tot['op wr-a '+ str(j)][i]/current,2)
        op_te[i] = op_te[i] + round(df_tot['op te-a '+ str(j)][i]/current,2)
df_tot['av op qb-a'] = op_qb
df_tot['av op rb-a'] = op_rb
df_tot['av op wr-a'] = op_wr
df_tot['av op te-a'] = op_te

#Find Opponent's Opponents Avg-Allowed each week (1,2,3...)
op_qb = [0]*32
op_rb = [0]*32
op_wr = [0]*32
op_te = [0]*32

for j in range(1,current+1):
    for i in range(0,32):
        op_qb[i] = df_tot['av op qb-a'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op op qb-a '+ str(j)] = op_qb
        op_rb[i] = df_tot['av op rb-a'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op op rb-a '+ str(j)] = op_rb
        op_wr[i] = df_tot['av op wr-a'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op op wr-a '+ str(j)] = op_wr
        op_te[i] = df_tot['av op te-a'][df_tot['Team'] == schedule[str(j)][i]].item()
        df_tot['op op te-a '+ str(j)] = op_te
       
#Calculate Mean Avg-Allowed for all opponent's opponents
op_qb = [0]*32
op_rb = [0]*32
op_wr = [0]*32
op_te = [0]*32
for j in range(1,current+1):
    for i in range(0,32):
        op_qb[i] = op_qb[i] + round(df_tot['op op qb-a '+ str(j)][i]/current,2)
        op_rb[i] = op_rb[i] + round(df_tot['op op rb-a '+ str(j)][i]/current,2)
        op_wr[i] = op_wr[i] + round(df_tot['op op wr-a '+ str(j)][i]/current,2)
        op_te[i] = op_te[i] + round(df_tot['op op te-a '+ str(j)][i]/current,2)
df_tot['av op op qb-a'] = op_qb
df_tot['av op op rb-a'] = op_rb
df_tot['av op op wr-a'] = op_wr
df_tot['av op op te-a'] = op_te

#Calculate "Expected" Points Allowed using Strength-of-Schedule formula
#Expected = (2*Op.Avg.Scored + Op.Op.Avg.Against)/3
df_tot['Expected Avg QB-A'] = round(2*df_tot['av op qb-s']/3 + df_tot['av op op qb-a']/3,2)
df_tot['Expected Avg RB-A'] = round(2*df_tot['av op rb-s']/3 + df_tot['av op op rb-a']/3,2)
df_tot['Expected Avg WR-A'] = round(2*df_tot['av op wr-s']/3 + df_tot['av op op wr-a']/3,2)
df_tot['Expected Avg TE-A'] = round(2*df_tot['av op te-s']/3 + df_tot['av op op te-a']/3,2)

#Calculate Global (league-wide) Averages for each position
#glob_qb = sum(df_tot['Av QB-A'])/32
#glob_rb = sum(df_tot['Av RB-A'])/32
#glob_wr = sum(df_tot['Av WR-A'])/32
#glob_te = sum(df_tot['Av TE-A'])/32

#Calculate Defensive Performance Rank
#Performance = (Expected - Actual)/Global Avg.
#Positive Value means the defense held the opposing team below the expected output (GOOD)
#Negative Value means the defense allowed greater than expected output (BAD)
df_tot['Rank QB'] = round(df_tot['Av QB-A'] - df_tot['Expected Avg QB-A'],4)
df_tot['Rank RB'] = round(df_tot['Av RB-A'] - df_tot['Expected Avg RB-A'],4)
df_tot['Rank WR'] = round(df_tot['Av WR-A'] - df_tot['Expected Avg WR-A'],4)
df_tot['Rank TE'] = round(df_tot['Av TE-A'] - df_tot['Expected Avg TE-A'],4)

print('Trimming Data...')
df_trim = df_tot[['Team','Av QB-A','Av RB-A', 'Av WR-A', 'Av TE-A','Rank QB','Rank RB','Rank WR','Rank TE']].copy()
#print(df_trim)

print('Complete')

desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
df_trim.to_csv(os.path.join(desktop, 'output.csv'))


What is the most recently completed Week of the NFL Schedule?...2
Begin gathering data...
Extracting data...
Final Calculations...
Trimming Data...
Complete
